In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json

c:\Users\Gebruiker\anaconda3\envs\thesis1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with open('data_mave/data_in_mave_0.json', 'r') as datafile:
    data = json.load(datafile)
    data = pd.DataFrame(data)

data

#data = pd.read_json('data_in_mave_0.json', encoding = 'utf-8-sig')

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
data['title embeddings'] = ''

a = 0
for row in data['title']:
    title_embed = model.encode(row)
    data.at[a, 'title embeddings'] = title_embed
    a += 1

In [ ]:
data

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

data['keys'] = ''
data['values'] = ''
data['title embeddings'] = ''

i = 0
for row in data['specs']:
    row_keys = list(row.keys())
    row_values = list(row.values())
    data.at[i, 'keys'] = row_keys
    data.at[i, 'values'] = row_values
    i += 1

a = 0
for row in data['title']:
    title_embed = model.encode(row)
    data.at[a, 'title embeddings'] = title_embed
    a += 1

In [ ]:
data_basic = data_basic.drop('specs', axis=1)

In [ ]:
from sklearn.cluster import DBSCAN

# Cluster title embeddings
title_embeds = []

for row in data['title embeddings']:
    title_embeds.append(row)

db = DBSCAN(eps=8, min_samples=5).fit(title_embeds)
title_labels = db.labels_


titles = data['title']
specs = data['specs']
data_new = list(zip(titles, specs,  title_labels))
sorted_titles = pd.DataFrame(data_new, columns=['titles', 'specs', 'cluster_num'])
sorted_titles = sorted_titles.sort_values(by=['cluster_num'])
sorted_titles.to_csv('clusters/sorted_titles_mave.csv')
sorted_titles


# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(title_labels)) - (1 if -1 in title_labels else 0)
n_noise_ = list(title_labels).count(-1)

print("Estimated number of title clusters: %d" % n_clusters_)
print("Estimated number of title noise points: %d" % n_noise_)

In [ ]:
titles = data['title']
keys = data['keys']
data = list(zip(titles, keys, title_labels))
sorted_titles = pd.DataFrame(data, columns=['titles', 'keys', 'cluster_num'])
sorted_titles = sorted_titles.sort_values(by=['cluster_num'])
sorted_titles.to_csv('clusters/sorted_titles_mave.csv')
sorted_titles

## DBSCAN over 10 first MAE dataset files

In [2]:
res = pd.DataFrame()

In [3]:
import os

files = os.listdir('data/train')
index = 0
for i in range(10):
    filename = files[i]
    print(filename)
    df = pd.read_json(f'data/train/{filename}')
    df = df.drop(['tokens', 'diffbotUri', 'text', 'images'], axis=1)

    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    df['keys'] = ''
    df['values'] = ''
    df['title embeddings'] = ''

    i = 0
    for row in df['specs']:
        row_keys = list(row.keys())
        row_values = list(row.values())
        df.at[i, 'keys'] = row_keys
        df.at[i, 'values'] = row_values
        i += 1

    a = 0
    for row in df['title']:
        title_embed = model.encode(row)
        df.at[a, 'title embeddings'] = title_embed
        a += 1


    df = df.drop('specs', axis=1)
    res = res.append(df)
    print(f'df shape{i}:{df.shape}')
    print(f'res shape{i}:{res.shape}')
    index += 1

dataset-000.json
df shape2173:(2173, 4)
res shape2173:(2173, 4)
dataset-001.json
df shape2351:(2351, 4)
res shape2351:(4524, 4)
dataset-002.json
df shape2281:(2281, 4)
res shape2281:(6805, 4)
dataset-003.json
df shape2260:(2260, 4)
res shape2260:(9065, 4)
dataset-004.json
df shape2314:(2314, 4)
res shape2314:(11379, 4)
dataset-005.json
df shape2161:(2161, 4)
res shape2161:(13540, 4)
dataset-006.json
df shape2332:(2332, 4)
res shape2332:(15872, 4)
dataset-007.json
df shape2224:(2224, 4)
res shape2224:(18096, 4)
dataset-008.json
df shape2305:(2305, 4)
res shape2305:(20401, 4)
dataset-009.json
df shape2264:(2264, 4)
res shape2264:(22665, 4)


In [10]:
res = res.drop_duplicates(subset='title')
res.shape

(19210, 4)

In [11]:
from sklearn.cluster import DBSCAN

# Cluster title embeddings (combined 10 MAE data parts)
title_embeds = []

for row in res['title embeddings']:
    title_embeds.append(row)

db = DBSCAN(eps=5, min_samples=5).fit(title_embeds)
title_labels = db.labels_

titles = res['title']
keys = res['keys']
df_new = list(zip(titles, keys,  title_labels))
sorted_titles = pd.DataFrame(df_new, columns=['titles', 'keys', 'cluster_num'])
sorted_titles = sorted_titles.sort_values(by=['cluster_num'])
sorted_titles.to_csv('clusters/sorted_titles_mae10.csv')
sorted_titles


# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(title_labels)) - (1 if -1 in title_labels else 0)
n_noise_ = list(title_labels).count(-1)

print("Estimated number of title clusters: %d" % n_clusters_)
print("Estimated number of title noise points: %d" % n_noise_)

Estimated number of title clusters: 46
Estimated number of title noise points: 11456
